In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os

# Load the MediaPipe Hands model
mp_hands = mp.solutions.hands.Hands()

def get_landmarks(file_location):
    # Read the image
    image = cv2.imread(file_location)

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and get the landmarks data
    results = mp_hands.process(image_rgb)

    # Extract the landmarks from the results
    landmarks = results.multi_hand_landmarks
    
    # Convert the landmarks to a numpy array
    if landmarks:
        landmarks_np = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks[0].landmark])
    else:
        return None
    
    return landmarks_np

Fontconfig warning: ignoring UTF-8: not a valid region tag
Matplotlib is building the font cache; this may take a moment.
I0000 00:00:1719476704.533810 10744552 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1719476704.555814 10746741 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719476704.562407 10746741 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [2]:
# count the number of subfolders and files in the given 
files = []
folders = []
def count_files(path='.'):
    global file_count, folder_count
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                files.append(entry.path)
            elif entry.is_dir():
                folders.append(entry.path)
                count_files(entry.path)
        
    return files, folders

count_files("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

len(set(files)), len(set(folders))

(223075, 29)

In [ ]:
# Initialize the list to store the landmarks
landmarks_list = []
        
# Initialize the list to store the labels
labels = []

global curr_label
curr_label = ""

def getLandmarksAndLabels(path='.'):
    global curr_label
    with os.scandir(path) as entries:
        
        for entry in entries:
            # print(entry.path)
            # print(curr_label)
            if entry.is_file():
                landmarks = get_landmarks(entry.path)
                # print(landmarks)
                if landmarks is not None:
                    landmarks_list.append(landmarks)
                    labels.append(curr_label)
                # else:
                #     print("No landmarks found")
            elif entry.is_dir():
                curr_label = entry.name
                print(curr_label)
                getLandmarksAndLabels(entry.path)
        
    return landmarks_list, labels

x_train, y_train = getLandmarksAndLabels("/Users/gvssriram/Desktop/projects-internship/MathNoteX/sign/ASL_Alphabet_Dataset/asl_alphabet_train")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1719360603.128705 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719360603.134692 9646481 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/gvssriram/anaconda3/envs/math_note_x/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


R
U
I
N
G
Z
T
S
A
F
O
H
del
nothing
space
M
J
C
D
V
Q
X
E
B
K
L
Y
P
W


In [ ]:
type(x_train), type(y_train), type(x_train[0]), type(y_train[0])

(list, list, numpy.ndarray, str)

In [ ]:
# convert the list to numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

((169090, 21, 3), (169090,))

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train.npy', x_train)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/y_train.npy', y_train)

In [ ]:
# reshape x_train
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_train.shape

(169090, 63)

In [ ]:
np.save('../data/ASL_Alphabet_Dataset_Processed/x_train_flat.npy', x_train)

## Load processed numpy image data

In [11]:
# Load numpy arrays
x_train = np.load('../data/ASL_Alphabet_Dataset_Processed/x_train_flat.npy')
y_train = np.load('../data/ASL_Alphabet_Dataset_Processed/y_train.npy')

In [12]:
x_train.shape, y_train.shape

((169090, 63), (169090,))

In [13]:
# one hot encode the labels
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.reshape(-1,1)).toarray()

In [14]:
y_train.shape

(169090, 29)

In [16]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
# Create a classification model using tensorflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Dense(128, input_shape=(x_train.shape[1],), activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(29, activation='softmax'))

model.summary()

opt = tf.keras.optimizers.Adam(learning_rate=0.005)

loss = tf.keras.losses.CategoricalCrossentropy()

metrics = []
metrics.append("categorical_accuracy")

# callbacks

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint(filepath='../models/ckpts/asl_alphabet_model.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
tensorboard = TensorBoard(log_dir='../tflogs')

callbacks = [early_stopping, model_checkpoint, reduce_lr, tensorboard]

model.compile(optimizer=opt, loss=loss, metrics=metrics)

history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=1, callbacks=callbacks)

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_62 (Dense)                │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 29)             │           957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,381 (79.61 KB)

 Trainable params: 19,933 (77.86 KB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 4s 842us/step - categorical_accuracy: 0.3985 - loss: 2.4395 - val_categorical_accuracy: 0.0076 - val_loss: 10.3009 - learning_rate: 0.0050
Epoch 2/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 788us/step - categorical_accuracy: 0.5157 - loss: 1.8057 - val_categorical_accuracy: 0.0231 - val_loss: 12.6858 - learning_rate: 0.0050
Epoch 3/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 757us/step - categorical_accuracy: 0.5357 - loss: 1.7636 - val_categorical_accuracy: 0.0297 - val_loss: 15.3775 - learning_rate: 0.0050
Epoch 4/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 755us/step - categorical_accuracy: 0.5387 - loss: 1.7373 - val_categorical_accuracy: 0.0296 - val_loss: 17.1316 - learning_rate: 0.0050
Epoch 5/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 762us/step - categorical_accuracy: 0.6112 - loss: 1.4563 - val_categorical_accuracy: 0.0284 - val_loss: 18.8104 - learning_rate: 1.0000e-03
Epoch 6/100
4228/4228 ━━━━━━━━━━━━━━━━━━━━ 3s 755us/step - categorical_accuracy: 0.633

In [53]:
y_pred = model.predict(x_train)

5285/5285 ━━━━━━━━━━━━━━━━━━━━ 1s 256us/step


In [54]:
y_pred[0]

array([2.2267817e-05, 1.4718435e-02, 1.6494931e-04, 6.1403662e-05,
       3.4717115e-04, 9.2047808e-04, 9.3395880e-04, 1.8891878e-02,
       4.9796313e-06, 3.6329898e-04, 2.9005447e-09, 3.0947418e-09,
       5.7257053e-05, 5.1436808e-05, 2.5897718e-05, 3.0631182e-09,
       6.4807863e-04, 2.5968230e-01, 1.5553079e-03, 3.2233997e-04,
       4.7521397e-01, 2.2304481e-01, 3.2933618e-09, 1.5384683e-03,
       3.0208964e-09, 4.6150794e-04, 1.7743702e-04, 2.4696965e-06,
       7.8986294e-04], dtype=float32)

In [55]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([20, 17, 17, ..., 21, 21, 20])

In [56]:
y_train_labels = np.argmax(y_train, axis=1)
y_train_labels

array([17, 17, 17, ..., 22, 22, 22])

In [57]:
# Evaluate the model

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

accuracy_score(y_train_labels, y_pred)

0.6685197232243184

In [58]:
print(classification_report(y_train_labels, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.92      0.78      6087
           1       0.86      0.96      0.91      6931
           2       0.96      0.81      0.88      5740
           3       0.56      0.79      0.66      6890
           4       0.85      0.88      0.87      6240
           5       0.89      0.96      0.93      7514
           6       0.80      0.93      0.86      6334
           7       0.75      0.85      0.79      6407
           8       0.64      0.90      0.75      6700
           9       0.50      0.87      0.63      5767
          10       0.00      0.00      0.00      7097
          11       0.00      0.00      0.00      7294
          12       0.66      0.64      0.65      4123
          13       0.74      0.58      0.65      3579
          14       0.86      0.88      0.87      6254
          15       0.00      0.00      0.00      5224
          16       0.80      0.90      0.85      5223
          17       0.46    

/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gvssriram/anaconda3/envs/sign_lang_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [ ]:
# Release the MediaPipe Hands model
mp_hands.close()